# NLP HW 1 

## Дарья Родионова БКЛ182

### Importing packages

In [33]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from collections import Counter
from time import sleep
from sklearn.metrics import accuracy_score
import nltk
import time
import random
import requests

from tqdm.auto import tqdm

from sklearn.model_selection import train_test_split

In [34]:
from nltk.tokenize import WordPunctTokenizer
from pymorphy2 import MorphAnalyzer

t = WordPunctTokenizer()
m = MorphAnalyzer()

In [35]:
session = requests.session()
ua = UserAgent(verify_ssl=False)

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/urllib/request.py", line 1350, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1255, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1301, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1250, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/http/client.py", line 1010, in _send_output
    self.send(msg)
  

### Getting movies

In [36]:
movies_dict = []

def get_movies():
    url = f'https://www.afisha.ru/msk/schedule_cinema/vybor-afishi/'
    response = session.get(url, headers={'User-Agent':ua.random})
    time.sleep(random.randint(3,10))
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    
    for m in soup.find_all('h3', {'class':'heHLK'}):
        movies_dict.append(m.find('a')['href'])
    return movies_dict

In [37]:
movies = get_movies()

Функция ниже скачивает отзывы про один фильм и записывает их в словарь. 

In [43]:
def get_pos_reviews(page_number):
    url = f'https://afisha.ru{page_number}reviews/positive'
    for _ in range(3): #update number of pages if necessary
        response = session.get(url)
        time.sleep(random.randint(3,10))
        
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('div', class_ ='restrict-text review__text')
    all_reviews = [review.text for review in reviews]
    return all_reviews

In [44]:
reviews_pos = []

for p in tqdm(movies):
    pos_words = get_pos_reviews(p)
    reviews_pos.extend(pos_words)

In [45]:
len(reviews_pos)

301

Аналогично с отрицательными отзывами. 

In [46]:
def get_neg_reviews(page_number):
    url = f'https://afisha.ru{page_number}reviews/negative'
    for _ in range(3):
        response = session.get(url)
        time.sleep(random.randint(3,10))

    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('div', class_ ='restrict-text review__text')
    all_reviews = [review.text for review in reviews]
    return all_reviews

In [49]:
reviews_neg = []

for p in tqdm(movies):
    neg_words = get_neg_reviews(p)
    reviews_neg.extend(neg_words)

In [50]:
len(reviews_neg)

115

### Spliting data 

In [159]:
def lemmatize(review):
    words = []
    text = t.tokenize(review[1:].lower())
    for word in text:
        if word.isalpha():
            lemmas = m.parse(word)[0].normal_form
            words.append(lemmas)
    return words

In [160]:
def split_data(pos, neg):
    pos_train, pos_test = train_test_split(pos)
    neg_train, neg_test = train_test_split(neg)
    
    test_data = {}
    
    for word in pos_test:
        test_data[word] = 1
    for word in neg_test:
        test_data[word] = 0
    return pos_train, neg_train, test_data

In [161]:
pos_train, neg_train, test_data = split_data(reviews_pos, reviews_neg)

### Classyfing reviews

Функция принимает лемматизированные положительные и отрицательные отзывы и помещает их в два отдельных списка. 

In [26]:
def classify_review(pos, neg, freq):
    pos_reviews = []
    neg_reviews = []
    for word in pos:
        pos_reviews.extend(lemmatize(word))
    for word in neg:
        neg_reviews.extend(lemmatize(word))
    
    pos_freq = Counter(pos_reviews)
    neg_freq = Counter(neg_reviews)
    
    pos_reviews = set(pos_reviews)
    neg_reviews = set(neg_reviews)
    
    common = pos_reviews.intersection(neg_reviews)
    
    pos_reviews.symmetric_difference_update(common)
    neg_reviews.symmetric_difference_update(common)
    
    pos_reviews = [word for word in pos_reviews if pos_freq[word] > freq]
    neg_reviews = [word for word in neg_reviews if neg_freq[word] > freq]
    return pos_reviews, neg_reviews

In [30]:
positive, negative = classify_review(pos_train, neg_train, 5)

### Prediction

In [45]:
def prediction(text, pos, neg):    
    words = lemmatize(text)
    
    pos_score = 0
    neg_score = 0
    
    for word in words:
        if word in pos:
            pos_score += 1
        elif word in neg:
            neg_score += 1
        else:
            pass
    
    return pos_score, neg_score

In [67]:
for text in test_data.keys(): # just to see whether everyting is working or not
    pos_score, neg_score = prediction(text, positive, negative)
    if pos_score > neg_score:
        print('Positive.')
    elif neg_score > pos_score:
        print('Negative.')
    else:
        print('Not clear.')
    print('Answer:', test_data[text], '\n')

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Positive.
Answer: 1 

Not clear.
Answer: 1 

Not clear.
Answe

### Accuracy 

In [71]:
predicted = []
gold = []

for text in test_data.keys():
    pos_score, neg_score = prediction(text, positive, negative)
    if pos_score > neg_score:
        predicted.append(1)
    else:
        predicted.append(0)

    if test_data[text] == 1:
        gold.append(1)
    else:
        gold.append(0)

print("Accuracy: %.4f" % accuracy_score(predicted, gold))

Accuracy: 0.5613


### Improvements

Самое простое и очевидное решение: увеличить количество данных. </p>

Кроме того, было бы неплохо сбалансировать выборку (положительных отзывов получилось в два раза больше, чем отрицательных). Ещё можно использовать векторы с tf-idf вместо тональных словарей.


### HW 2 (продолжение)

<p> Лучший результат показал PyMoprhy, но Natasha удобнее в использовании, поэтому буду использовать именно её. <p>

<p> Возьмём следующие биграммы и триграммы:  <p>

1. "Не" + прилагательное (добавление частицы меняет положительную оценку на отрицательную.)
2. "Не" + наречие ("не плохо")
4. Прилагательное + существительное (стандартное описание понятий, "увлекательный сюжет", "замечательная игра")
5. Наречие + глагол (отзывы типа "плохо играют")
6. "Не" + наречие + глагол ("не очень играют")

In [153]:
from natasha import(
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [154]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()
doc = Doc(rus_text)

Функция, выделяющая n-граммы.

In [155]:
def find_ngrams(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    n_grams = []
    for i in doc.tokens:
        i.lemmatize(morph_vocab)
        pos_tags = [(i.lemma, i.pos) for i in doc.tokens]
        
    for i in range(len(pos_tags)):
        if (i+1) != len(pos_tags):
            if pos_tags[i][0] == 'не':
                if pos_tags[i+1][1] == 'ADJ':
                    n_grams.append(' '.join([pos_tags[i][0], pos_tags[i+1][0]]))
                elif pos_tags[i+1][1] == 'ADV':
                    if pos_tags[i+2][1] == 'VERB':
                        n_grams.append(' '.join([pos_tags[i][0], pos_tags[i+1][0], pos_tags[i+2][0]]))
            elif pos_tags[i][1] == 'ADJ':
                if pos_tags[i+1][1] == 'NOUN':
                    n_grams.append(' '.join([pos_tags[i][0], pos_tags[i+1][0]]))
            elif pos_tags[i][1] == 'ADV':
                if pos_tags[i+1][1] == 'VERB':
                    n_grams.append(' '.join([pos_tags[i][0], pos_tags[i+1][0]]))
    
    return n_grams

(Та же самая функция лемматизации, что и выше, просто перенесла сюда для удобства)

In [156]:
def lemmatizer(review):
    words = []
    text = t.tokenize(review[1:].lower())
    for word in text:
        if word.isalpha():
            lemmas = m.parse(word)[0].normal_form
            words.append(lemmas)
    return words

In [166]:
def classify_review_ngrams(pos, neg, freq):
    pos_reviews = []
    neg_reviews = []
    for word in pos:
        pos_reviews.extend(lemmatizer(word))
        pos_reviews.extend(find_ngrams(word)) # now with groups
    for word in neg:
        neg_reviews.extend(lemmatizer(word))
        neg_reviews.extend(find_ngrams(word)) # same here
    
    pos_freq = Counter(pos_reviews)
    neg_freq = Counter(neg_reviews)
    
    pos_reviews = set(pos_reviews)
    neg_reviews = set(neg_reviews)
    
    common = pos_reviews.intersection(neg_reviews)
    
    pos_reviews.symmetric_difference_update(common)
    neg_reviews.symmetric_difference_update(common)
    
    pos_reviews = [word for word in pos_reviews if pos_freq[word] > freq]
    neg_reviews = [word for word in neg_reviews if neg_freq[word] > freq]
    return pos_reviews, neg_reviews

In [158]:
positive, negative = classify_review_ngrams(pos_train, neg_train, 2)

In [167]:
def prediction_ngrams(text, pos, neg):  
    words = lemmatizer(text)
    words.extend(find_ngrams(text)) # updated
    
    pos_score = 0
    neg_score = 0
    
    for word in words:
        if word in pos:
            pos_score += 1
        elif word in neg:
            neg_score += 1
        else:
            pass
    
    return pos_score, neg_score

In [171]:
predicted = []
gold = []

for text in test_data.keys():
    pos_score, neg_score = prediction_ngrams(text, positive, negative) # updated
    if pos_score > neg_score:
        predicted.append(1)
    else:
        predicted.append(0)

    if test_data[text] == 1:
        gold.append(1)
    else:
        gold.append(0)

In [172]:
len(predicted), len(gold)

(105, 105)

In [173]:
print('Accuracy: %.4f' % accuracy_score(predicted, gold))

Accuracy: 0.9238


0.5613 → 0.9238! Качество заметно улучшилось. 